In [36]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta

In [37]:
input_workbook = 'Input.xlsx'

In [38]:
df = pd.read_excel(input_workbook)

In [39]:
df_MD = pd.read_excel(input_workbook,sheet_name="Mod")
df_YTM = pd.read_excel(input_workbook,sheet_name="YTM")
df_AM = pd.read_excel(input_workbook,sheet_name="Average Maturity")
df_NAV = pd.read_excel(input_workbook,sheet_name="NAV")
df_monthlyNAV = pd.read_excel(input_workbook,sheet_name="Monthly NAV")

In [40]:
start_date = input('Enter the date from which you want to run the analysis (in YYYY-MM-DD format): ')
year, month, day = map(int, start_date.split('-'))
start_date = datetime.datetime(year, month, day)
end_date = input('Enter the date till which you want to run the analysis (in YYYY-MM-DD format): ')
year, month, day = map(int, end_date.split('-'))
end_date = datetime.datetime(year, month, day)

Enter the date from which you want to run the analysis (in YYYY-MM-DD format): 2021-01-01
Enter the date till which you want to run the analysis (in YYYY-MM-DD format): 2022-09-30


In [41]:
Dates = pd.date_range(start_date,end_date,freq='d')
Dates

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24',
               '2022-09-25', '2022-09-26', '2022-09-27', '2022-09-28',
               '2022-09-29', '2022-09-30'],
              dtype='datetime64[ns]', length=638, freq='D')

In [42]:
schemes_list = list(df.columns.values)
schemes_list.remove('Date')
schemes_list

['Aditya Birla Sun Life Money Manager Fund - Dir - Growth',
 'HDFC Money Market Fund - Dir - Growth',
 'ICICI Prudential Money Market Fund - Dir- Growth',
 'Invesco India Money Market Fund - Dir - Growth',
 'Kotak Money Market Fund - Dir - Growth',
 'UTI Money Market Fund -Dir- Growth']

In [43]:
n = len(schemes_list) - 2
n

4

In [44]:
oneday = pd.Timedelta(days=1)
oneday

Timedelta('1 days 00:00:00')

In [45]:
i=0
dn = pd.DataFrame()
while i <=n:
    scheme = schemes_list[i]
    df1 = pd.DataFrame ({
        'Scheme': scheme,
        'Date': Dates,
    })
    temp = pd.DataFrame()
    temp = pd.merge(df1,df_MD[['Date',scheme]],on='Date',how='left')
    temp = temp.fillna(method='ffill')
    temp.rename(columns={scheme :'Mod on Purchase date'},inplace=True)
    temp = pd.merge(temp,df_YTM[['Date',scheme]],on='Date',how='left')
    temp = temp.fillna(method='ffill')
    temp.rename(columns={scheme :'YTM - Previous Month end'},inplace=True)
    temp = pd.merge(temp,df_monthlyNAV[['Date',scheme]],on='Date',how='left')
    temp = temp.fillna(method='ffill')
    temp.rename(columns={scheme :'Nav - Previous Month end'},inplace=True)
    temp = pd.merge(temp,df_NAV[['Date',scheme]],on='Date',how='left')
    temp = temp.fillna(method='ffill')
    temp.rename(columns={scheme :'Nav - Purchase date'},inplace=True)
    temp = pd.merge(temp,df_AM[['Date',scheme]],on='Date',how='left')
    temp = temp.fillna(method='ffill')
    temp.rename(columns={scheme :'Avg Mat'},inplace=True)
    temp['Nav on end date'] = temp['Nav - Purchase date'][temp['Date']==end_date]
    temp = temp.fillna(method='bfill')
    temp['YTM as on end date'] = temp['YTM - Previous Month end'][temp['Date']==end_date]
    temp = temp.fillna(method='bfill')
    from pandas.tseries.offsets import MonthBegin
    temp['MS'] = temp['Date'].dt.normalize().map(MonthBegin().rollback)
    temp.rename(columns={'MS' :'Month Start'},inplace=True)
    dn = pd.concat([dn,temp],axis=0)
    i = i+1

In [46]:
#Missing Data
dn[(dn['Mod on Purchase date'] == '--') | (dn['YTM - Previous Month end'] == '--')\
   | (dn['Nav - Previous Month end'] == '--') | (dn['Nav - Purchase date'] == '--') | (dn['Avg Mat'] == '--') ]

,Scheme,Date,Mod on Purchase date,YTM - Previous Month end,Nav - Previous Month end,Nav - Purchase date,Avg Mat,Nav on end date,YTM as on end date,Month Start


In [47]:
#Current Slippage

dn['Return'] = (dn['Nav - Purchase date']/dn['Nav - Previous Month end']-1)*100
dn['Return-Est'] = dn['YTM - Previous Month end']/100*((dn['Date']-dn['Month Start'])/oneday/365)*100
dn['Yield impact'] = (dn['Return-Est'] - dn['Return'])/dn['Mod on Purchase date']
dn['Yield on Purchase date'] = dn['Yield impact']+(dn['YTM - Previous Month end']/100)*100
dn['IRR as on end date'] = (((dn['Nav on end date']/dn['Nav - Purchase date'])**((1/((end_date-dn['Date'])/365/oneday))))-1)*100
dn['Slippage (Current)'] = dn['IRR as on end date'] - dn['Yield on Purchase date']

In [48]:
#Slippage at Avg. Maturity

dn1=pd.DataFrame()

dn1['Purchase Date'] = dn['Date']


dn1['Maturity Date as per purchase date'] = dn.apply\
(lambda x: x['Date'] + relativedelta(months = int(x['Avg Mat']*12)), axis = 1)

dn1['Days left from end date'] = np.where(((dn1['Maturity Date as per purchase date']-end_date)/oneday)>0,\
(dn1['Maturity Date as per purchase date']-end_date)/oneday,0)

dn1['Nav on maturity'] = dn['Nav on end date']*((1+dn['YTM as on end date']/100)**((dn1['Days left from end date'])/365))

dn1['IRR at current YTM'] = (((dn1['Nav on maturity']/dn['Nav - Purchase date'])**\
                            ((1/((dn1['Maturity Date as per purchase date']-dn['Date'])/365/oneday))))-1)*100

dn1['Slippage (at Avg. Mat.)'] = np.where((dn1['Days left from end date'])==0,\
'Already Matured',(dn1['IRR at current YTM']-dn['Yield on Purchase date']))

dn1.set_index('Purchase Date')

,Maturity Date as per purchase date,Days left from end date,Nav on maturity,IRR at current YTM,Slippage (at Avg. Mat.)
Purchase Date,,,,,
2021-01-01,2021-04-01,0.0,305.206300,33.003150,Already Matured
2021-01-02,2021-04-02,0.0,305.206300,33.003150,Already Matured
2021-01-03,2021-04-03,0.0,305.206300,33.003150,Already Matured
2021-01-04,2021-04-04,0.0,305.206300,32.750132,Already Matured
2021-01-05,2021-04-05,0.0,305.206300,32.635940,Already Matured
...,...,...,...,...,...
2022-09-26,2023-01-26,118.0,3778.591854,6.857490,-0.5034093181633841
2022-09-27,2023-01-27,119.0,3779.273940,6.990068,-0.4735279943571875
2022-09-28,2023-01-28,120.0,3779.956149,6.998260,-0.472975398406831


In [49]:
#Slippage at 3 years

dn2 = pd.DataFrame()

dn2['Purchase Date'] = dn['Date']

dn2['Date of completing 3 years'] = dn.apply\
(lambda x: x['Date']+relativedelta(years = 3), axis = 1)

dn2['Days left from 3y completion'] = np.where(((dn2['Date of completing 3 years']-end_date)/oneday)>0,\
(dn2['Date of completing 3 years']-end_date)/oneday,0)

dn2['Nav on maturity'] = dn['Nav on end date']*((1+dn['YTM as on end date']/100)**((dn2['Days left from 3y completion'])/365))

dn2['IRR at 3y'] = (((dn2['Nav on maturity']/dn['Nav - Purchase date'])**\
                            ((1/((dn2['Date of completing 3 years']-dn['Date'])/365/oneday))))-1)*100

dn2['Slippage (at 3y)'] = np.where((dn2['Days left from 3y completion'])==0,\
'Already Matured',(dn2['IRR at 3y']-dn['Yield on Purchase date']))

dn2.set_index('Purchase Date')

,Date of completing 3 years,Days left from 3y completion,Nav on maturity,IRR at 3y,Slippage (at 3y)
Purchase Date,,,,,
2021-01-01,2024-01-01,458.0,332.249317,5.310247,1.360247467480633
2021-01-02,2024-01-02,459.0,332.310911,5.316755,1.3329361607642163
2021-01-03,2024-01-03,460.0,332.372516,5.323262,1.3056252561309662
2021-01-04,2024-01-04,461.0,332.434132,5.313287,1.4085901444919076
2021-01-05,2024-01-05,462.0,332.495760,5.312345,1.4401789094481288
...,...,...,...,...,...
2022-09-26,2025-09-26,1092.0,4504.853617,6.815285,-0.5456142689328596
2022-09-27,2025-09-27,1093.0,4505.666802,6.830029,-0.6335669884034063
2022-09-28,2025-09-28,1094.0,4506.480135,6.830940,-0.6402959871763683


In [50]:
df_slippage = pd.DataFrame ({
    'Purchase Date' : dn['Date'],
    'Scheme': dn['Scheme'],
    'Current Slippage' : dn['Slippage (Current)'],
    'Slippage at avg. maturity': dn1['Slippage (at Avg. Mat.)'],
    'Slippage at 3Y': dn2['Slippage (at 3y)']
}).set_index('Purchase Date')

In [51]:
with pd.ExcelWriter('MultiScheme.xlsx') as writer:
    dn.to_excel(writer, sheet_name='Mapping & Current Slippage Cal')
    dn1.to_excel(writer, sheet_name='Avg. maturity')
    dn2.to_excel(writer, sheet_name='At 3Y')
    df_slippage.to_excel(writer, sheet_name='Slippage')